In [10]:
%pip install -r ../requirements.txt
path = '../path.txt'
with open(path, 'r') as archivo:
    for linea in archivo.readlines():
        exec(linea.strip(), globals())


  Using cached asttokens-2.4.1-py2.py3-none-any.whl (27 kB)
  Using cached branca-0.7.2-py3-none-any.whl (25 kB)
  Using cached comm-0.2.2-py3-none-any.whl (7.2 kB)
  Using cached contourpy-1.2.1-cp311-cp311-win_amd64.whl (188 kB)
  Using cached debugpy-1.8.1-cp311-cp311-win_amd64.whl (4.7 MB)
  Using cached executing-2.0.1-py2.py3-none-any.whl (24 kB)
  Using cached fonttools-4.51.0-cp311-cp311-win_amd64.whl (2.2 MB)
  Using cached ipykernel-6.29.4-py3-none-any.whl (117 kB)
  Using cached ipython-8.24.0-py3-none-any.whl (816 kB)
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
  Using cached jupyter_client-8.6.1-py3-none-any.whl (105 kB)
  Using cached jupyter_core-5.7.2-py3-none-any.whl (28 kB)
  Using cached matplotlib-3.8.4-cp311-cp311-win_amd64.whl (7.7 MB)
  Using cached matplotlib_inline-0.1.7-py3-none-any.whl (9.9 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl (5.2 kB)
  Using cached numpy-1.26.4-cp311-cp311

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\ANGEL\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Árbol de decisión

Este modelo busca predecir el cargo que tiene un servicio depéndiendo del trafico
* congestion_surcharge = 0 ---> sin trafico
* congestion_surcharge = 2 ---> con trafico

Se importan las librerias requeridas

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from joblib import dump

Se hace el entrenamiento con 3 datasets de recorridos de taxis ecologicos para tres meses consecutivos del año 2023, se concatenan y se usan para el entrenamiento

In [12]:
d0 = pd.read_parquet(df0)
df1 = pd.read_parquet(df1)
df2 = pd.read_parquet(df2)
df = pd.concat([d0, df1, df2])

In [13]:
df.shape

(183020, 12)

Se filtra el data set con los valores de la columna 'congestion_surcharge' mayor que -1 ya que -1 representa a los valores nulos

In [14]:
df = df[df['congestion_surcharge'] > -1]
df.shape

(169725, 12)

In [15]:
df['congestion_surcharge'].value_counts()

congestion_surcharge
0    120670
2     49055
Name: count, dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169725 entries, 0 to 59302
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   start_trip            169725 non-null  datetime64[us]
 1   end_trip              169725 non-null  datetime64[us]
 2   pu_location_id        169725 non-null  int64         
 3   do_location_id        169725 non-null  int64         
 4   passenger_count       169725 non-null  int64         
 5   trip_distance         169725 non-null  float64       
 6   fare_amount           169725 non-null  float64       
 7   tip_amount            169725 non-null  float64       
 8   total_amount          169725 non-null  float64       
 9   payment_type          169725 non-null  int64         
 10  congestion_surcharge  169725 non-null  int64         
 11  type_of_taxi          169725 non-null  int64         
dtypes: datetime64[us](2), float64(4), int64(6)
memory usage: 16.8 MB

In [17]:
df.head(1)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi
0,2023-10-01 00:57:33,2023-10-01 01:07:58,166,74,1,1.45,12.1,2.92,17.52,1,0,1


Se extrate informacion de las fechas de inicio y fin de viaje para generar columnas con los siguientes valores: año, mes, dia y minutos de viaje

In [18]:
df['year'] = df['start_trip'].dt.year
df['month'] = df['start_trip'].dt.month
df['day'] = df['start_trip'].dt.dayofweek
df['min_trip'] = df['min_trip'] = (df['end_trip'] - df['start_trip']).dt.total_seconds() / 60
df.sample(5)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi,year,month,day,min_trip
18921,2023-12-10 02:26:50,2023-12-10 02:46:18,25,249,1,4.20,22.6,5.55,33.40,1,2,1,2023,12,6,19.466667
7783,2023-11-04 15:15:12,2023-11-04 15:30:52,188,188,1,1.40,14.9,0.00,16.40,-1,0,1,2023,11,5,15.666667
12175,2023-12-06 22:11:09,2023-12-06 22:17:16,97,181,1,1.05,7.9,2.08,12.48,1,0,1,2023,12,2,6.116667
55734,2023-10-29 00:35:59,2023-10-29 00:52:36,7,173,2,3.92,21.2,0.00,23.70,-1,0,1,2023,10,6,16.616667
11528,2023-11-06 16:32:39,2023-11-06 16:58:16,166,262,1,3.46,24.7,6.29,37.74,1,2,1,2023,11,0,25.616667


In [19]:
df.columns

Index(['start_trip', 'end_trip', 'pu_location_id', 'do_location_id',
       'passenger_count', 'trip_distance', 'fare_amount', 'tip_amount',
       'total_amount', 'payment_type', 'congestion_surcharge', 'type_of_taxi',
       'year', 'month', 'day', 'min_trip'],
      dtype='object')

In [20]:
df.shape

(169725, 16)

In [21]:
# Arbol de decision

In [22]:
from sklearn import tree
# Selección de las variables independientes
X = df[['min_trip', 'trip_distance', 'pu_location_id', 'do_location_id']]

# Conversión de las variables categóricas a dummies
X = pd.get_dummies(X)

# Selección de la variable dependiente
y = df['congestion_surcharge']

# Separación del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Creación del clasificador de árbol de decisión
arbol = tree.DecisionTreeClassifier(max_depth = 7, random_state=42)

# Entrenamiento del modelo
arbol.fit(X_train, y_train)

# Evaluación del modelo
score = arbol.score(X_test, y_test)

# Predicción del recargo por congestión
predicciones = arbol.predict(X_test)

# Impresión del score del modelo
print(f"Score del modelo: {score}")

# Impresión de las primeras 10 predicciones
#print(f"Primeras 10 predicciones: {predicciones[:10]}")

# Visualización del árbol de decisión
#tree.plot_tree(arbol)

Score del modelo: 0.9321738310708899


In [23]:
resultados = pd.DataFrame({'Valor real': y_test, 'Valor predicho': predicciones}).head(5)

# Impresión del dataframe
print(resultados.to_string())

       Valor real  Valor predicho
24431           2               2
42188           0               0
3614            0               0
38539           0               0
36135           0               0


Se observa un Score de 0.93, y se visualiza una columna con valores reales y valores predichos

Se guarda el modelo en formato joblib, para ser posteriormente consumido por Streamlit

In [26]:
dump(arbol, modelo_cargo_entrenado)

['../modelos_ML/modelos_entrenados/modelo_cargo_entrenado.pkl']